In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from flask import Flask, request, jsonify

C:\Users\hugov\AppData\Local\Temp\ipykernel_2884\722869088.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Chargement et nettoyage des données
data = pd.read_csv('Partie 1\Housing.csv').dropna()
categorical_vars = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
data[categorical_vars] = data[categorical_vars].apply(lambda x: x.map({'yes': 1, 'no': 0}))
data['furnishingstatus'] = data['furnishingstatus'].map({'furnished': 1, 'semi-furnished': 0, 'unfurnished': 0})

In [3]:
# Remplacement des valeurs manquantes par la moyenne ou une valeur intermédiaire
for col in ['area', 'bedrooms', 'bathrooms', 'stories'] + categorical_vars:
    if data[col].isnull().any():
        if data[col].dtype == np.number:
            data[col].fillna(data[col].mean(), inplace=True)
        else:
            data[col].fillna(0.5, inplace=True)  # pour les variables catégorielles avec des valeurs manquantes

In [4]:
# Préparation des ensembles de données d'entraînement et de test
X = data.drop('price', axis=1)
y = data['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Entraînement du modèle linéaire
model = LinearRegression().fit(X_train, y_train)

In [6]:
# Évaluation du modèle
predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
print(f"Coefficient de détermination R2 : {r2}")

Coefficient de détermination R2 : 0.6419909931610206


In [7]:
# Création de l'application Flask pour l'accès via API
app = Flask(__name__)

# Utilisez seulement le chemin relatif après l'adresse de base et le port
@app.route('/predict', methods=['GET'])
def predict():
    try:
        features = ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'furnishingstatus']
        input_data = np.array([[float(request.args.get(feature)) for feature in features]])
        prediction = model.predict(input_data)[0]

        return jsonify({
            'prediction': prediction,
            'message': 'Prédiction du prix réussie.'
        })

    except Exception as e:
        return jsonify({
            'error': str(e),
            'message': 'Erreur lors de la prédiction du prix.'
        })

if __name__ == '__main__':
    # Lorsque vous lancez l'application, Flask utilise 127.0.0.1:5000 par défaut pour le développement local
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
